<a href="https://colab.research.google.com/github/Maximus125/Predicting_Loan/blob/master/Loan_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## *Reading the data*

In [27]:
df = pd.read_csv('loan_data.csv')
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


## *Looking into the data*

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [29]:
df.shape

(614, 13)

## *Dividing the data into imput features and target variables*

In [30]:
y = df['Loan_Status']
X = df.drop('Loan_Status', axis=1)

In [31]:
print(X.shape)
print(y.shape)

(614, 12)
(614,)


## *Now splitting the dataset into train_set and test_set*

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [33]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(491, 12)
(123, 12)
(491,)
(123,)


## *From now on we will work only on training set*
## *Lets check for Null Values*

In [34]:
print(X_train.isnull().sum())
print(y_train.isnull().sum())

Loan_ID               0
Gender               10
Married               2
Dependents           11
Education             0
Self_Employed        28
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           20
Loan_Amount_Term     11
Credit_History       43
Property_Area         0
dtype: int64
0


In [35]:
# So some features of training set has indeed Null Values
# Some features are Numerical whereas some are Categorical
# We need to deal with it accordingly

In [36]:
X_train['ApplicantIncome'].fillna(X_train['ApplicantIncome'].median(), inplace=True)
X_train['CoapplicantIncome'].fillna(X_train['CoapplicantIncome'].median(), inplace=True)
X_train['LoanAmount'].fillna(X_train['LoanAmount'].median(), inplace=True)
X_train['Loan_Amount_Term'].fillna(X_train['Loan_Amount_Term'].median(), inplace=True)
X_train['Credit_History'].fillna(X_train['Credit_History'].median(), inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [37]:
X_train.isnull().sum()
# Looking at the numerical features we see there's no null values now

Loan_ID               0
Gender               10
Married               2
Dependents           11
Education             0
Self_Employed        28
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History        0
Property_Area         0
dtype: int64

In [38]:
# Lets do the same with categorical features
X_train['Gender'].fillna(X_train['Gender'].mode()[0], inplace=True)
X_train['Married'].fillna(X_train['Married'].mode()[0], inplace=True)
X_train['Dependents'].fillna(X_train['Dependents'].mode()[0], inplace=True)
X_train['Self_Employed'].fillna(X_train['Self_Employed'].mode()[0], inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [39]:
X_train.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

In [40]:
# So no Null Values in our training input set

## *Changing Categorical Features into Numerical Features*

In [43]:
X_train['Gender'] = X_train['Gender'].map({'Male':0, 'Female':1})     

#Doing the same thing for every other Categorical features

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
X_train['Married'] = X_train['Married'].map({'No':0, 'Yes':1})
X_train['Dependents'] = X_train['Dependents'].map({'0':0, '1':1, '2':2, '3+':3})
X_train['Education'] = X_train['Education'].map({'Graduate':1, 'Not Graduate':0})
X_train['Self_Employed'] = X_train['Self_Employed'].map({'No':0, 'Yes':1})
X_train['Property_Area'] = X_train['Property_Area'].map({'Rural':0, 'Semiurban':1, 'Urban':2})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [46]:
X_train.head()

#Categorical features for X_train has been converted to numerical

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
260,LP001865,0,1,1,1,0,6083,4250.0,330.0,360.0,1.0,2
303,LP001977,0,1,1,1,0,1625,1803.0,96.0,360.0,1.0,2
407,LP002314,1,0,0,0,0,2213,0.0,66.0,360.0,1.0,0
261,LP001868,0,0,0,1,0,2060,2209.0,134.0,360.0,1.0,1
352,LP002141,0,1,3,1,0,2666,2083.0,95.0,360.0,1.0,0


In [55]:
y_train = y_train.map({'Y':1, 'N':0})

#Categorical feature in the y_train is now converted into numerical

In [56]:
y_train

260    1
303    1
407    1
261    1
352    1
      ..
369    0
320    1
527    1
125    1
265    1
Name: Loan_Status, Length: 491, dtype: int64